# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [39]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop


In [40]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [41]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [42]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [43]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [44]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [45]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, None, 50)          1000000   
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 5)                 280       
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [46]:
rmsprop = RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [47]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 8s 10ms/step - loss: 0.6752 - accuracy: 0.5388 - val_loss: 0.6433 - val_accuracy: 0.6329
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.6216 - accuracy: 0.6995 - val_loss: 0.6152 - val_accuracy: 0.7284
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.5935 - accuracy: 0.7457 - val_loss: 0.5971 - val_accuracy: 0.7355
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.5721 - accuracy: 0.7661 - val_loss: 0.5838 - val_accuracy: 0.7417
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.5532 - accuracy: 0.7794 - val_loss: 0.5689 - val_accuracy: 0.7531
Epoch 6/10
782/782 [==============================] - 7s 10ms/step - loss: 0.5368 - accuracy: 0.7890 - val_loss: 0.5578 - val_accuracy: 0.7539
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.5220 - accuracy: 0.7971 - val_loss: 0.5458 - val_accuracy: 0.7641
Epoc

In [48]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 1s 2ms/step - loss: 0.5280 - accuracy: 0.7648
Test score: 0.5279865264892578
Test accuracy: 0.7648000121116638


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [49]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [50]:
x_train

array([[   15,   256,     4, ...,    19,   178,    32],
       [  125,    68,     2, ...,    16,   145,    95],
       [  645,   662,     8, ...,     7,   129,   113],
       ...,
       [  529,   443, 17793, ...,     4,  3586,     2],
       [  286,  1814,    23, ...,    12,     9,    23],
       [   97,    90,    35, ...,   204,   131,     9]])

In [51]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [52]:
rmsprop = RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [53]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 14s 17ms/step - loss: 0.5986 - accuracy: 0.6676 - val_loss: 0.5105 - val_accuracy: 0.7456
Epoch 2/10
782/782 [==============================] - 13s 17ms/step - loss: 0.4480 - accuracy: 0.7928 - val_loss: 0.4501 - val_accuracy: 0.7880
Epoch 3/10
782/782 [==============================] - 13s 16ms/step - loss: 0.3833 - accuracy: 0.8306 - val_loss: 0.4137 - val_accuracy: 0.8080
Epoch 4/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3415 - accuracy: 0.8524 - val_loss: 0.3946 - val_accuracy: 0.8186
Epoch 5/10
782/782 [==============================] - 13s 16ms/step - loss: 0.3137 - accuracy: 0.8658 - val_loss: 0.3921 - val_accuracy: 0.8240
Epoch 6/10
782/782 [==============================] - 13s 16ms/step - loss: 0.2935 - accuracy: 0.8776 - val_loss: 0.3740 - val_accuracy: 0.8323
Epoch 7/10
782/782 [==============================] - 13s 16ms/step - loss: 0.2781 - accuracy: 0.8851 - val_loss: 0.3681 - val_accuracy:

In [54]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 100  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [55]:
rnn_hidden_dim = 5
word_embedding_dim = 100
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [56]:
rmsprop = RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [57]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 17s 21ms/step - loss: 0.5567 - accuracy: 0.7141 - val_loss: 0.4642 - val_accuracy: 0.7861
Epoch 2/10
782/782 [==============================] - 15s 19ms/step - loss: 0.4083 - accuracy: 0.8173 - val_loss: 0.4095 - val_accuracy: 0.8115
Epoch 3/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3495 - accuracy: 0.8495 - val_loss: 0.3733 - val_accuracy: 0.8324
Epoch 4/10
782/782 [==============================] - 14s 18ms/step - loss: 0.3185 - accuracy: 0.8646 - val_loss: 0.3556 - val_accuracy: 0.8408
Epoch 5/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2993 - accuracy: 0.8751 - val_loss: 0.3532 - val_accuracy: 0.8442
Epoch 6/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2852 - accuracy: 0.8829 - val_loss: 0.3414 - val_accuracy: 0.8482
Epoch 7/10
782/782 [==============================] - 16s 21ms/step - loss: 0.2758 - accuracy: 0.8856 - val_loss: 0.3416 - val_accuracy:

In [58]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2522 - accuracy: 0.8970 - val_loss: 0.3357 - val_accuracy: 0.8528
Epoch 2/10
782/782 [==============================] - 15s 19ms/step - loss: 0.2489 - accuracy: 0.8990 - val_loss: 0.3357 - val_accuracy: 0.8525
Epoch 3/10
782/782 [==============================] - 15s 19ms/step - loss: 0.2454 - accuracy: 0.9010 - val_loss: 0.3447 - val_accuracy: 0.8508
Epoch 4/10
782/782 [==============================] - 15s 19ms/step - loss: 0.2422 - accuracy: 0.9017 - val_loss: 0.3442 - val_accuracy: 0.8524
Epoch 5/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2391 - accuracy: 0.9028 - val_loss: 0.3436 - val_accuracy: 0.8515
Epoch 6/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2354 - accuracy: 0.9063 - val_loss: 0.3529 - val_accuracy: 0.8475
Epoch 7/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2328 - accuracy: 0.9076 - val_loss: 0.3585 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation